# Hello World! Inference - Bring Your Own Pickle File 'Model'

In [1]:
!pip install dill

In [2]:
import dill as pickle

### Create our Custom Hello World 'Model'
Here we create a python class called Model and inside it we have a method called predict that takes an integer and returns "Hello World!" that many times.

In [3]:
class Model:
    def predict(x): 
        if isinstance(x, (int)):  # Check if x is a number
            sentence = "Hello, World! " * x  # Repeat "Hello, World!" x times
            return sentence  # Print the sentence
        else:
            print("Error: input must be an integer")  # Display an error message if x is not a number

### Pickle the Model. 
Now we want to serialize our custom 'Model'. 

In [4]:
with open('hello-world-model.pkl', 'wb') as f:
    pickle.dump(Model, f)

### Load the model and test
Now we can deserialize the model and test it.

In [5]:
with open('hello-world-model.pkl', 'rb') as f:
    model = pickle.load(f)

#### Predict locally

In [6]:
import pandas as pd

raw_data = 2

model.predict(raw_data)

'Hello, World! Hello, World! '

### Package our Model to deploy to a SageMaker endpoint
SageMaker requires our Model to be tared and gzipped. 

In [7]:
! tar -czvf hello_world_model.tar.gz hello-world-model.pkl

hello-world-model.pkl


### Upload our Model to S3
Now we can upload our Model pickeled (serialized) Model to S3

In [8]:
import sagemaker

bucket = sagemaker.Session().default_bucket()
prefix = 'DEMO-hello-world'

    
! aws s3 cp hello_world_model.tar.gz s3://$bucket/$prefix/model.tar.gz

upload: ./hello_world_model.tar.gz to s3://sagemaker-us-east-1-171503325295/DEMO-hello-world/model.tar.gz


### Build and Push our container to ECR
We have our custom Model that is now in S3. All we need now is a container that implemenets the hosting requirements and inference logic.
An important file to look at is the predictor.py here we coded the logic to deserialize the Model and make a inference from it. SageMaker fetched our Model from S3 and placed it in /opt/ml/model/. Take a look at the get_model() method which uses the code above to load the model from the SageMaker model path. 

In [9]:
!sed -n '26,31p' container/Files/predictor.py

    def get_model(cls):
        """Get the model object for this instance, loading it if it's not already loaded."""
        if cls.model == None:
            with open(os.path.join(model_path, 'hello-world-model.pkl'), 'rb') as f:
                cls.model = pickle.load(f)
        return cls.model


In [10]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-hello-world-inference

cd container


chmod +x Files/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded
Sending build context to Docker daemon  18.43kB
Step 1/9 : FROM python:3.10-slim-buster
 ---> 93b9055430ce
Step 2/9 : RUN pip install --upgrade pip
 ---> Using cache
 ---> 202699474e01
Step 3/9 : RUN apt-get update && apt-get install -y     wget     nginx     ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> a8bcacb11fbf
Step 4/9 : RUN pip3 install flask gevent gunicorn pandas dill
 ---> Using cache
 ---> e2a0855d8700
Step 5/9 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> f09967b6f1d0
Step 6/9 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> 9075a508c39a
Step 7/9 : ENV PATH="/opt/program:${PATH}"
 ---> Using cache
 ---> a1631373de7b
Step 8/9 : COPY Files /opt/program
 ---> Using cache
 ---> 229f7565b5d3
Step 9/9 : WORKDIR /opt/program
 ---> Using cache
 ---> add57f4634f8
Successfully built add57f4634f8
Successfully tagged sagemaker-hello-world-inference:latest
The push refers to repository [171503325295.dkr.ecr.us-east-1.amazonaw

### Deploy our Model to an Endpoint
Our container has been pushed to ECR and our Model is in S3 now we have everything we need to Deploy to a SageMaker Endpoint.

#### Create a SageMaker Model

In [11]:
from sagemaker.model import Model
from sagemaker.local import LocalSession
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

from sagemaker import get_execution_role
role = get_execution_role()

In [12]:
class PredictorRealTime(Predictor):
    def __init__(self, endpoint_name, sagemaker_session=None):
        super(PredictorRealTime, self).__init__(
            endpoint_name, 
            sagemaker_session, 
            serializer=JSONSerializer(),
            deserializer=JSONDeserializer())

In [13]:
sagemaker_local_session = LocalSession()
sagemaker_session = sagemaker.Session()
account = sagemaker_session.boto_session.client('sts').get_caller_identity()['Account']
region = sagemaker_session.boto_session.region_name

image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-hello-world-inference:latest'.format(account, region)

sagemaker_local_model = Model(
                    sagemaker_session= sagemaker_local_session,
                    model_data = "s3://"+bucket+"/"+prefix+"/model.tar.gz" , 
                    image_uri= image,
                    role=role,
                    predictor_cls=PredictorRealTime)

#### Deploy locally to test container

In [14]:
local_predictor = sagemaker_local_model.deploy(
                                initial_instance_count= 1,
                                instance_type= 'local')

Attaching to 5au3s64jny-algo-1-izfv3
5au3s64jny-algo-1-izfv3  | Starting the inference server with 8 workers.
5au3s64jny-algo-1-izfv3  | [2023-12-28 23:35:56 +0000] [10] [INFO] Starting gunicorn 21.2.0
5au3s64jny-algo-1-izfv3  | [2023-12-28 23:35:56 +0000] [10] [INFO] Listening at: unix:/tmp/gunicorn.sock (10)
5au3s64jny-algo-1-izfv3  | [2023-12-28 23:35:56 +0000] [10] [INFO] Using worker: gevent
5au3s64jny-algo-1-izfv3  | [2023-12-28 23:35:56 +0000] [12] [INFO] Booting worker with pid: 12
5au3s64jny-algo-1-izfv3  | [2023-12-28 23:35:56 +0000] [13] [INFO] Booting worker with pid: 13
5au3s64jny-algo-1-izfv3  | [2023-12-28 23:35:56 +0000] [14] [INFO] Booting worker with pid: 14
5au3s64jny-algo-1-izfv3  | [2023-12-28 23:35:56 +0000] [15] [INFO] Booting worker with pid: 15
5au3s64jny-algo-1-izfv3  | [2023-12-28 23:35:56 +0000] [23] [INFO] Booting worker with pid: 23
5au3s64jny-algo-1-izfv3  | [2023-12-28 23:35:56 +0000] [24] [INFO] Booting worker with pid: 24
5au3s64jny-algo-1-izfv3  | [20

#### Get a prediction locally

In [15]:
input_json = {
    
"SayHelloWorld": 3
    
}

result = local_predictor.predict(input_json)
result

5au3s64jny-algo-1-izfv3  | result:
5au3s64jny-algo-1-izfv3  | {'SayHelloWorldResults': 'Hello, World! Hello, World! Hello, World! '}
5au3s64jny-algo-1-izfv3  | 172.18.0.1 - - [28/Dec/2023:23:36:30 +0000] "POST /invocations HTTP/1.1" 200 70 "-" "python-urllib3/1.26.18"


{'SayHelloWorldResults': 'Hello, World! Hello, World! Hello, World! '}

#### Deploy to a SageMaker Endpoint

In [16]:
sagemaker_model = Model(
                    sagemaker_session= sagemaker_session,
                    model_data = "s3://"+bucket+"/"+prefix+"/model.tar.gz" , 
                    image_uri= image,
                    role=role,
                    predictor_cls=PredictorRealTime)

predictor = sagemaker_model.deploy(
                                initial_instance_count= 1,
                                instance_type= 'ml.t2.medium')

-----!

#### Get a prediction from our Endpoint

In [17]:
input_json = {
    
"SayHelloWorld": 100
    
}

result = predictor.predict(input_json)
result

{'SayHelloWorldResults': 'Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, World! Hello, W

#### Optional cleanup
When you're done with the endpoint, you'll want to clean it up.

In [ ]:
sagemaker_local_session.delete_endpoint(local_predictor.endpoint)
sagemaker_session.delete_endpoint(predictor.endpoint)